# Fake address gen

In [1]:
import pandas as pd
import json
import re
import random
import string as string_lib

## Word list for fake address generation

Taken from <https://github.com/duyet/vietnamese-wordlist>, using the 11K words file.

In [2]:
with open("./InputData/Viet11K.txt", 'r', encoding="utf-8") as file:
    vn_words = file.read().splitlines()
vn_words

['a',
 'a dua',
 'a hoàn',
 'a phiến',
 'a tòng',
 'à',
 'ả',
 'ả đào',
 'ả giang hồ',
 'á',
 'á-căn-đình',
 'á khẩu',
 'á khôi',
 'á kim',
 'á rập',
 'ạ',
 'ác',
 'ác cảm',
 'ác chiến',
 'ác mộng',
 'ác nghiệt',
 'ác phụ',
 'ác tà',
 'ác tâm',
 'ác thú',
 'ách',
 'ạch',
 'ai',
 'ai ai',
 'ai điếu',
 'ai oán',
 'ải',
 'ải quan',
 'ái',
 'ái ân',
 'ái hữu',
 'ái khanh',
 'ái lực',
 'ái mộ',
 'ái nam ái nữ',
 'ái ngại',
 'ái nhĩ lan',
 'ái phi',
 'ái quốc',
 'ái tình',
 'am',
 'am hiểu',
 'am pe',
 'ảm đạm',
 'ám',
 'ám ảnh',
 'ám chỉ',
 'ám hại',
 'ám hiệu',
 'ám muội',
 'ám sát',
 'ám tả',
 'ám thị',
 'an',
 'an ba ni',
 'an bài',
 'an cư',
 'an dưỡng',
 'an nghỉ',
 'an ninh',
 'an phận',
 'an táng',
 'an tâm',
 'an toàn',
 'an ủi',
 'an vị',
 'án',
 'án mạng',
 'án ngữ',
 'án phí',
 'án sát',
 'án thư',
 'án tử hình',
 'ang',
 'ang áng',
 'áng',
 'anh',
 'anh ánh',
 'anh dũng',
 'anh đào',
 'anh em',
 'anh hùng',
 'anh linh',
 'anh tài',
 'anh thư',
 'anh tuấn',
 'ảnh',
 'ảnh ảo',
 'ả

## Import and adminstrative region data

In [3]:
with open("./InputData/region_tree.json", 'r') as file:
    region_tree = json.load(file)
    
with open("./InputData/reverse_index_dist_to_prov.json", 'r') as file:
    rev_index_dist_to_prov = json.load(file)
    
with open("./InputData/reverse_index_ward_to_prov.json", 'r') as file:
    rev_index_ward_to_prov = json.load(file)
    
with open("./InputData/reverse_index_ward_to_dist.json", 'r') as file:
    rev_index_ward_to_dist = json.load(file)

## String deterioration function

In [4]:
def expand_space_within_string(string: str, limit: (int, int), /, 
                               include_newline=False, newline_prob=0.05, 
                               include_tab=False, tab_prob=0.05) -> str:
    tokens = string.split()
    newline_prob = (newline_prob if include_newline else 0)
    tab_prob = (tab_prob if include_tab else 0)
    
    space_prob = 1 - newline_prob - tab_prob
    prob_weight = [space_prob, newline_prob, tab_prob]
    whitespace = [' ', '\n', '\t']
    result = str()
    
    for i, token in enumerate(tokens):
        result += token
        if i == len(tokens):
            pass
        num_space = random.randrange(*limit)
        spaces = ''.join(random.choices(whitespace, prob_weight, k=num_space))
        result += spaces
    
    return result

In [5]:
test = 'this is a test string'
expand = expand_space_within_string(
    test, (2, 8), 
    include_newline=True, newline_prob=0.1, 
    include_tab=True, tab_prob=0.05)
print(expand)

this     is    a   
  
test   string  


In [6]:
# by default, string will deteriorate by having their chars substituted
def substitute_string(string:str) -> str:
    position = random.randrange(0, len(string))
    replace_with = string_lib.ascii_letters + string_lib.digits + string_lib.punctuation
    
    return string[:position] + random.choice(replace_with) + string[position+1:]
    
    
def insert_string(string:str) -> str:
    position = random.randrange(0, len(string))
    insert_with = string_lib.ascii_letters + string_lib.digits + string_lib.punctuation
    
    return string[:position] + random.choice(insert_with) + string[position:]


def delete_string(string:str) -> str:
    position = random.randrange(0, len(string))
    
    return string[:position] + string[position+1:]


def transpose_string(string:str) -> str:
    if len(string) < 2:
        return string
    
    position = random.randrange(0, len(string) - 1)
    
    return string[:position] + string[position+1] + string[position] + string[position+2:]
 
    
# substitution always on by default    
def deteriorate_string(string:str, edit_distance=1, /,
                      allow_insertion=True, allow_deletion=True, allow_transposition=True, 
                      insert_prob=0.1, delete_prob=0.1, trans_prob=0.05) -> str:
    
    insert_prob = (insert_prob if allow_insertion else 0)
    delete_prob = (delete_prob if allow_deletion else 0)
    trans_prob = (trans_prob if allow_transposition else 0)
    
    sub_prob = 1 - insert_prob - delete_prob - trans_prob
    action_weights = [sub_prob, insert_prob, delete_prob, trans_prob]
    actions = [substitute_string, insert_string, delete_string, transpose_string]
    
    for _ in range(edit_distance):
        action = random.choices(actions, action_weights, k=1)[0]
        string = action(string)
        
    return string

In [7]:
print(substitute_string('test'))
print(insert_string('test'))
print(delete_string('test'))
print(transpose_string('test'))

te=t
teVst
est
tset


In [8]:
deteriorate_string('this is a test string, it should be destroy incrementally', 6,
                  allow_insertion=True, allow_deletion=True, allow_transposition=True)

'this is a nest((tring, it Xhould be d:s1troy incrementally'

## Abbreviation

In [9]:
def get_abbre_with_word_first_letter(string: str, /, with_sep=False, sep='.') -> str:
    
    words = string.strip().split()
    if len(words) < 2:
        return string
    
    for i, word in enumerate(words):
        words[i] = word.capitalize()
    
    sep = sep if with_sep else ''
    result = ''
    
    for word in words:
        if not word.isalnum():
            result += word[0]
        else:
            result += word[0] + (sep if with_sep else '')
    
    return result

In [10]:
print(get_abbre_with_word_first_letter('Hồ Chí Minh - Đà Nẵng'))
print(get_abbre_with_word_first_letter('Hồ Chí Minh - Đà Nẵng', with_sep=True))

HCM-ĐN
H.C.M.-Đ.N.


In [11]:
def get_abbre_with_word_first_letter_incomplete_last_word(string: str, /, 
                                                          with_sep=False, sep='.',
                                                          with_space_for_last_word=False) -> str:
    words = string.strip().split()
    if len(words) < 2:
        return string
    
    for i, word in enumerate(words):
        words[i] = word.capitalize()
    
    sep = sep if with_sep else ''
    result = ''
    
    for word in words[:-1]:
        if not word.isalnum():
            result += word[0]
        else:
            result += word[0] + (sep if with_sep else '')
    
    result += (' ' if with_space_for_last_word else '') + words[-1]
    
    return result

In [12]:
print(get_abbre_with_word_first_letter_incomplete_last_word('Hồ Chí Minh - Đà Nẵng'))
print(get_abbre_with_word_first_letter_incomplete_last_word('Hồ Chí Minh - Đà Nẵng', with_sep=True))

HCM-ĐNẵng
H.C.M.-Đ.Nẵng


In [13]:
def get_abbre(string: str, /, full_abbr_prop=0.3, incomplete_abbr_prop=0.7, 
              with_sep_prob=0.6, with_space_for_last_word_prop=0.3) -> str:
    
    abbre_actions = [get_abbre_with_word_first_letter, get_abbre_with_word_first_letter_incomplete_last_word]
    abbre_actions_weight = [full_abbr_prop, incomplete_abbr_prop]
    action = random.choices(abbre_actions, abbre_actions_weight, k=1)[0]
    
    with_sep = random.random() < with_sep_prob
    with_space_for_last_word = random.random() < with_space_for_last_word_prop
    
    if action == get_abbre_with_word_first_letter_incomplete_last_word:
        result = action(string, with_sep=with_sep, with_space_for_last_word=with_space_for_last_word)
    else:
        result = action(string, with_sep=with_sep)
        
    #print(string, '-->', result)
    return result

In [14]:
print(get_abbre('Hồ Chí Minh - Đà Nẵng'))
print(get_abbre('Hồ Chí Minh - Đà Nẵng'))

HCM-ĐNẵng
H.C.M.-Đ.N.


## Fake address generation

In [15]:
def get_prov_dist_ward_level_region(region_tr) -> {'province': {}, 'district': {}, 'ward': {}}:
    random_prov_info = random.choice(list(region_tr.values()))
    random_dist_info = random.choice(list(random_prov_info['districts'].values()))
    random_ward_info = random.choice(list(random_dist_info['wards'].values()))
    
    return {
        'province': random_prov_info, 
        'district': random_dist_info, 
        'ward': random_ward_info
    }

def get_prov_dist_ward_level_region_name(selection: {
        'province': {}, 
        'district': {}, 
        'ward': {}
    }) -> {'province': str, 'district': str, 'ward': str}:
    
    return {
        'province': selection['province']['name'], 
        'district': selection['district']['name'], 
        'ward': selection['ward']['name']
    }

In [16]:
get_prov_dist_ward_level_region(region_tree)

{'province': {'prefix': 'Tỉnh',
  'name': 'Bắc Kạn',
  'districts': {'Thành Phố Bắc Kạn': {'prefix': '',
    'name': 'Thành Phố Bắc Kạn',
    'wards': {'Nguyễn Thị Minh Khai': {'prefix': 'Phường',
      'name': 'Nguyễn Thị Minh Khai'},
     'Sông Cầu': {'prefix': 'Phường', 'name': 'Sông Cầu'},
     'Đức Xuân': {'prefix': 'Phường', 'name': 'Đức Xuân'},
     'Phùng Chí Kiên': {'prefix': 'Phường', 'name': 'Phùng Chí Kiên'},
     'Huyền Tụng': {'prefix': 'Phường', 'name': 'Huyền Tụng'},
     'Dương Quang': {'prefix': 'Xã', 'name': 'Dương Quang'},
     'Nông Thượng': {'prefix': 'Xã', 'name': 'Nông Thượng'},
     'Xuất Hóa': {'prefix': 'Phường', 'name': 'Xuất Hóa'}}},
   'Pác Nặm': {'prefix': 'Huyện',
    'name': 'Pác Nặm',
    'wards': {'Bằng Thành': {'prefix': 'Xã', 'name': 'Bằng Thành'},
     'Nhạn Môn': {'prefix': 'Xã', 'name': 'Nhạn Môn'},
     'Bộc Bố': {'prefix': 'Xã', 'name': 'Bộc Bố'},
     'Công Bằng': {'prefix': 'Xã', 'name': 'Công Bằng'},
     'Giáo Hiệu': {'prefix': 'Xã', 'name'

In [17]:
def generate_house_address(with_sublevel=True, sublevel=2) -> str:
    
    num_list = [str(random.randrange(1, 200)) for _ in range(sublevel)]
    result = str()
    for i in range(sublevel):
        result += num_list[i]
        if i < sublevel-1:
            result += '/'
    
    return result


def generate_street_name(min_name_word_len=3, add_prefix=True, abbr_prefix=False, abbr_name=False) -> str:
    
    name = str()
    name_word_len = 0
    while name_word_len < min_name_word_len:
        words = random.choice(vn_words).split()
        name_word_len += len(words)
        name += ' ' + ' '.join(word.capitalize() for word in words)
    
    name = get_abbre(name.strip()) if abbr_name else name.strip()
    prefix = get_abbre("Đường") if abbr_prefix else "Đường"
    
    if add_prefix:
        return prefix + ' ' + name
    else:
        return name

    
def generate_hamlet_name(min_name_word_len=2, add_prefix=True, abbr_prefix=False, abbr_name=False) -> str:
    
    prefixes = ["Thôn", "Xóm", "Ấp", "Đội"]
    
    choose_number = random.random() <= 0.5
    if not choose_number:
        name =  generate_street_name(min_name_word_len, add_prefix=False)
    else:
        name = str(random.randrange(1, 30))
    
    name = get_abbre(name) if abbr_name else name
    prefix = random.choice(prefixes)
    prefix = get_abbre(prefix) if abbr_prefix else prefix
    
    if add_prefix | choose_number:
        return prefix + ' ' + name
    else:
        return name

def generate_quarter_name(min_name_word_len=2, add_prefix=True, abbr_prefix=False, abbr_name=False) -> str:
    
    choose_number = random.random() <= 0.1
    if not choose_number:
        name =  generate_street_name(min_name_word_len, add_prefix=False)
    else:
        name = str(random.randrange(1, 30))
    
    name = get_abbre(name) if abbr_name else name
    prefix = "Khu phố"
    prefix = get_abbre(prefix) if abbr_prefix else prefix
    
    if add_prefix | choose_number:
        return prefix + ' ' + name
    else:
        return name
    
    
def generate_address_below_ward_level(add_house_address=True, # so nha
                                      add_street=True,        # ten duong
                                      add_hamlet=True,        # thon, xom, ap, doi
                                      add_quarter=True        # khu pho
                                     ) -> {
    'address': str,
    'street': str,
    'hamlet': str,
    'quarter': str
}:
    add_prefix_street = random.random() <= 0.5
    add_prefix_hamlet = random.random() <= 0.5
    add_prefix_quarter = random.random() <= 0.5
    
    abbre_params_street = {
        'abbr_prefix': random.random() <= 0.3, 
        'abbr_name': random.random() <= 0.1
    }
    abbre_params_hamlet = {
        'abbr_prefix': random.random() <= 0.3, 
        'abbr_name': random.random() <= 0.1
    }
    abbre_params_quarter = {
        'abbr_prefix': random.random() <= 0.3, 
        'abbr_name': random.random() <= 0.1
    }
    
    return {
        'address': generate_house_address() 
            if add_house_address else None,
        'street': generate_street_name(add_prefix=add_prefix_street, **abbre_params_street) 
            if add_street else None,
        'hamlet': generate_hamlet_name(add_prefix=add_prefix_hamlet, **abbre_params_hamlet) 
            if add_hamlet else None,
        'quarter': generate_quarter_name(add_prefix=add_prefix_quarter, **abbre_params_quarter) 
            if add_quarter else None
    }
    
    
def generate_address_from_ward_level_onward(ward, district, province,
                                            add_ward=True, add_dist=True, add_prov=True) -> {
    'ward': str,
    'district': str,
    'province': str
}:
    result = {    
        'ward': None,
        'district': None,
        'province': None
    }
    
    ward_name = ward['name']
    ward_prefix = ward['prefix']
    dist_name = district['name']
    dist_prefix = district['prefix']
    prov_name = province['name']
    prov_prefix = province['prefix']
    
    abbr_prefix = random.random() < 0.3
    abbr_name = random.random() < 0.1
    ward_name = get_abbre(ward_name) if abbr_name else ward_name
    ward_prefix = get_abbre(ward_prefix) if abbr_prefix else ward_prefix
    
    abbr_prefix = random.random() < 0.3
    abbr_name = random.random() < 0.1
    dist_name = get_abbre(dist_name) if abbr_name else dist_name
    dist_prefix = get_abbre(dist_prefix) if abbr_prefix else dist_prefix
    
    abbr_prefix = random.random() < 0.3
    abbr_name = random.random() < 0.1
    prov_name = get_abbre(prov_name) if abbr_name else prov_name
    prov_prefix = get_abbre(prov_prefix) if abbr_prefix else prov_prefix
    
    if add_ward:
        result['ward'] = (ward_prefix + ' ' if random.random() <= 0.5 else '') + ward_name
    if add_dist:
        result['district'] = (dist_prefix + ' ' if random.random() <= 0.5 else '') + dist_name
    if add_prov:
        result['province'] = (prov_prefix + ' ' if random.random() <= 0.5 else '') + prov_name
    
    return result
        

In [18]:
print(generate_house_address())
print(generate_street_name(min_name_word_len=3, abbr_prefix=random.random() < 0.5, abbr_name=random.random() < 0.5))
print(generate_hamlet_name(min_name_word_len=2, abbr_prefix=random.random() < 0.5, abbr_name=random.random() < 0.5))
print(generate_quarter_name(min_name_word_len=2, abbr_prefix=random.random() < 0.5, abbr_name=random.random() < 0.5))

71/9
Đường Tiếp Kiến Hiệp Định
Thôn 4
K.Phố XH


In [19]:
generate_address_below_ward_level()

{'address': '96/24',
 'street': 'T.G.T.C.',
 'hamlet': 'Thôn 10',
 'quarter': 'Lì Thị Sảnh'}

In [20]:
selection = get_prov_dist_ward_level_region(region_tree)
generate_address_from_ward_level_onward(**selection)

{'ward': 'Xã Cư Klông', 'district': 'Krông Năng', 'province': 'Tỉnh Đắk Lắk'}

In [21]:
def generate_fake_address(add_house_address=True, # so nha
                          add_street=True,        # ten duong
                          add_hamlet=True,        # thon, xom, ap, doi
                          add_quarter=True,       # khu pho
                          add_ward=True, add_dist=True, add_prov=True) -> (str, {}):
    
    below = generate_address_below_ward_level(
        add_house_address=add_house_address,
        add_street=add_street,
        add_hamlet=add_hamlet,
        add_quarter=add_quarter
    )
    
    selection = get_prov_dist_ward_level_region(region_tree)
    above = generate_address_from_ward_level_onward(
        **selection,
        add_ward=add_ward, add_dist=add_dist, add_prov=add_prov
    )
    
    combined = {**below, **above}
    
    part_list = []
    for addr_part in combined.values():
        if addr_part is not None:
            part_list.append(addr_part)
    
    return (', '.join(part_list), selection)

In [22]:
generate_fake_address()

('24/29, Bỡ Ngỡ Lưu Vong, Đội Lên Men, Khu phố Hồng Tâm, Thạnh Đức, Gò Dầu, Tỉnh Tây Ninh',
 {'province': {'prefix': 'Tỉnh',
   'name': 'Tây Ninh',
   'districts': {'Tây Ninh': {'prefix': 'Thành phố',
     'name': 'Tây Ninh',
     'wards': {'1': {'prefix': 'Phường', 'name': '1'},
      '3': {'prefix': 'Phường', 'name': '3'},
      '4': {'prefix': 'Phường', 'name': '4'},
      'Hiệp Ninh': {'prefix': 'Phường', 'name': 'Hiệp Ninh'},
      '2': {'prefix': 'Phường', 'name': '2'},
      'Thạnh Tân': {'prefix': 'Xã', 'name': 'Thạnh Tân'},
      'Tân Bình': {'prefix': 'Xã', 'name': 'Tân Bình'},
      'Bình Minh': {'prefix': 'Xã', 'name': 'Bình Minh'},
      'Ninh Sơn': {'prefix': 'Phường', 'name': 'Ninh Sơn'},
      'Ninh Thạnh': {'prefix': 'Phường', 'name': 'Ninh Thạnh'}}},
    'Tân Biên': {'prefix': 'Huyện',
     'name': 'Tân Biên',
     'wards': {'Tân Biên': {'prefix': 'Thị trấn', 'name': 'Tân Biên'},
      'Tân Lập': {'prefix': 'Xã', 'name': 'Tân Lập'},
      'Thạnh Bắc': {'prefix': 'Xã',

In [23]:
for i in range(10):
    param = {
        'add_house_address': random.random() <= 0.6,
        'add_street': random.random() <= 0.8,
        'add_hamlet': random.random() <= 0.5,
        'add_quarter': random.random() <= 0.5,
        'add_ward': random.random() <= 0.9,
        'add_dist': random.random() <= 0.9,
        'add_prov': random.random() <= 0.9,
    }
    addr, ground_truth = generate_fake_address(**param)
    addr = deteriorate_string(addr, 2)
    #addr = expand_space_within_string(addr, (1, 4), include_newline=True)
    print(i, ')--------------------')
    print(addr)
    print(get_prov_dist_ward_level_region_name(ground_truth))

0 )--------------------
Xã B Cường,)Định Hóh, Tỉnh Thái Nguyên
{'province': 'Thái Nguyên', 'district': 'Định Hóa', 'ward': 'Bảo Cường'}
1 )--------------------
79/183, Đường Chân Lý Hạ, Vĩnh Thịnh, Hoà Bình, Bcạ Liêu
{'province': 'Bạc Liêu', 'district': 'Hoà Bình', 'ward': 'Vĩnh Thịnh'}
2 )--------------------
39/156, Trừng Trị Ủ /ột, Đội 8, Trà Vỡ Lòng, Xã DiênXĐiền, Diên Khánh
{'province': 'Khánh Hòa', 'district': 'Diên Khánh', 'ward': 'Diên Điền'}
3 )--------------------
Đường Thể Tất H/a, Huyện Tràng ĐOnh, Lạng Sơn
{'province': 'Lạng Sơn', 'district': 'Tràng Định', 'ward': 'Kháng Chiến'}
4 )--------------------
121/99, Xóm 10, Khu phố Thi&n Mệnh, Thị trấn Đắk Mil, Huyện Đ.M., Tỉnh ĐắE Nông
{'province': 'Đắk Nông', 'district': 'Đắk Mil', 'ward': 'Đắk Mil'}
5 )--------------------
Đường Sòng Quần Xo, Long Hựu 'ây, Cần Đước, Long An
{'province': 'Long An', 'district': 'Cần Đước', 'ward': 'Long Hựu Tây'}
6 )--------------------
173/82,Nhị Trù)g Âm, Tạnh Ráo, Phúc Thành, Tỉnh Thái Bình
